In [19]:
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


In [9]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28*28))
x_test = np.reshape(x_test, (len(x_test), 28*28))
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [24]:
class RBM:
    def __init__(self, num_visible, num_hidden):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = tf.Variable(tf.random.normal([num_visible, num_hidden], stddev=0.1))
        self.visible_bias = tf.Variable(tf.zeros([num_visible]))
        self.hidden_bias = tf.Variable(tf.zeros([num_hidden]))

    def prob_hidden_given_visible(self, visible):
        return tf.nn.sigmoid(tf.matmul(visible, self.weights) + self.hidden_bias)

    def prob_visible_given_hidden(self, hidden):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(self.weights)) + self.visible_bias)

    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))

    def gibbs_sample(self, visible):
        hidden_prob = self.prob_hidden_given_visible(visible)
        hidden_sample = self.sample_prob(hidden_prob)
        visible_prob = self.prob_visible_given_hidden(hidden_sample)
        return visible_prob, hidden_sample

    def contrastive_divergence(self, input_data, learning_rate=0.1, k=1):
        positive_hidden_prob = self.prob_hidden_given_visible(input_data)

        # Gibbs Sampling
        chain_start = input_data
        for step in range(k):
            if step == 0:
                visible_prob, hidden_sample = self.gibbs_sample(chain_start)
            else:
                visible_prob, hidden_sample = self.gibbs_sample(visible_prob)
        
        negative_visible_prob = visible_prob

        # Compute gradients
        positive_associations = tf.matmul(tf.transpose(input_data), positive_hidden_prob)
        negative_associations = tf.matmul(tf.transpose(negative_visible_prob), hidden_sample)

        # Update parameters
        self.weights.assign_add(learning_rate * (positive_associations - negative_associations))
        self.visible_bias.assign_add(learning_rate * tf.reduce_mean(input_data - negative_visible_prob, 0))
        self.hidden_bias.assign_add(learning_rate * tf.reduce_mean(positive_hidden_prob - hidden_sample, 0))



In [11]:
# Example usage
num_visible = 784  # Number of pixels in MNIST images (28x28)
num_hidden = 256  # Number of hidden units in the RBM



In [25]:
rbm = RBM(num_visible, num_hidden)


In [15]:
for epoch in range(10):
    for i in range(len(x_train) // 128):
        batch_x = x_train[i*128:(i+1)*128]
        rbm.contrastive_divergence(batch_x)
        

In [16]:

# Extract features
x_train_features = tf.nn.sigmoid(tf.matmul(x_train, rbm.weights) + rbm.hidden_bias)
x_test_features = tf.nn.sigmoid(tf.matmul(x_test, rbm.weights) + rbm.hidden_bias)


In [21]:

# Training logistic regression on top of RBM features
logistic = LogisticRegression(max_iter=1000)  # Initialize logistic regression with default parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


In [23]:
# Fit the model to the training data
logistic.fit(x_train_features, np.argmax(y_train, axis=1))

# Predictions on training data
pred_train = logistic.predict(x_train_features)

# Predictions on test data
pred_test = logistic.predict(x_test_features)

# Evaluate model
accuracy_train = np.mean(pred_train == np.argmax(y_train, axis=1))
accuracy_test = np.mean(pred_test == np.argmax(y_test, axis=1))

print("Training Accuracy:", accuracy_train)
print("Test Accuracy:", accuracy_test)

Training Accuracy: 0.4282
Test Accuracy: 0.4239
